In [3]:
# step1: imporing dataset
import pandas as pd

# Load the dataset
csv_path = 'SmartPhones.csv'  
dataset = pd.read_csv(csv_path)

# Display the first few rows
dataset.head(10)


,Model Name,Review,Sentiment
0,Galaxy Z Fold 5,I?ve been using the Galaxy Z Fold 5 for about ...,Positive
1,iPhone 14 Pro,I upgraded to the iPhone 14 Pro expecting some...,Negative
2,Pixel 8,The Pixel 8 has been my daily driver for a few...,Neutral
3,OnePlus 12,The OnePlus 12 is hands-down the best smartpho...,Positive
4,Xiaomi 14,The Xiaomi 14 has been a complete disaster fro...,Negative
5,Moto Edge 50,The Moto Edge 50 has been my phone for a few w...,Neutral
6,Sony Xperia 1 VI,The Sony Xperia 1 VI is an absolute masterpiec...,Positive
7,Oppo Find X7,"The Oppo Find X7 has been a total nightmare, a...",Negative
8,Vivo X100,The Vivo X100 has been my phone for a couple o...,Neutral
9,Nokia G60,The Nokia G60 has completely surprised me in t...,Positive


In [4]:
# step 2: data pre-processing (NLP)
# a. Tokenization, stopword removal, lemmatization, lowerization, remove punctuations
import nltk
import string
from nltk.tokenize import word_tokenize
from nltk.corpus import stopwords
from nltk.stem import WordNetLemmatizer

# Initialize lemmatizer
lemmatizer = WordNetLemmatizer()
stop_words = set(stopwords.words('english'))

# Function for text cleaning
def preprocess_text(text):
    text = text.lower()  # Convert to lowercase
    text = text.translate(str.maketrans('', '', string.punctuation))  # Remove punctuation
    words = word_tokenize(text)  # Tokenization
    words = [lemmatizer.lemmatize(word) for word in words if word not in stop_words]  # Lemmatization & Stopword removal
    return ' '.join(words)

# Apply preprocessing to reviews
dataset['Cleaned_Review'] = dataset['Review'].astype(str).apply(preprocess_text)

# Display cleaned data
dataset[['Model Name', 'Cleaned_Review']].head(10)


,Model Name,Cleaned_Review
0,Galaxy Z Fold 5,ive using galaxy z fold 5 month honestly say o...
1,iPhone 14 Pro,upgraded iphone 14 pro expecting something rev...
2,Pixel 8,pixel 8 daily driver month perfectly fine phon...
3,OnePlus 12,oneplus 12 handsdown best smartphone ive ever ...
4,Xiaomi 14,xiaomi 14 complete disaster moment unboxed reg...
5,Moto Edge 50,moto edge 50 phone week perfectly average devi...
6,Sony Xperia 1 VI,sony xperia 1 vi absolute masterpiece ive obse...
7,Oppo Find X7,oppo find x7 total nightmare im counting day d...
8,Vivo X100,vivo x100 phone couple month solid device does...
9,Nokia G60,nokia g60 completely surprised best way possib...


In [5]:
# 2:b. extract attributes programmatically using frequent or common words extraction. GIve the attribute weights randomly for each one.

import nltk
from collections import Counter
from nltk.tokenize import word_tokenize
from nltk import pos_tag


# Tokenize and extract nouns/adjectives
def extract_nouns_adjectives(text):
    tokens = word_tokenize(text)
    pos_tags = pos_tag(tokens)
    keywords = [word for word, tag in pos_tags if tag in ['NN', 'JJ']]  # Nouns and Adjectives
    return keywords

# Apply extraction to all cleaned reviews
dataset['Keywords'] = dataset['Cleaned_Review'].apply(extract_nouns_adjectives)

# Flatten the list of keywords and find the most common ones
all_keywords = [word for keywords in dataset['Keywords'] for word in keywords]
common_words = Counter(all_keywords).most_common(50)

# Display the most common words
print(common_words)


[('ive', 54), ('phone', 36), ('feel', 23), ('day', 22), ('im', 18), ('camera', 16), ('use', 15), ('software', 13), ('call', 13), ('price', 12), ('time', 12), ('video', 11), ('battery', 11), ('quality', 11), ('design', 11), ('life', 10), ('full', 10), ('speaker', 10), ('update', 10), ('screen', 9), ('dont', 9), ('shot', 9), ('work', 9), ('feature', 9), ('good', 9), ('fingerprint', 9), ('sound', 9), ('display', 8), ('heavy', 8), ('bit', 8), ('nice', 8), ('issue', 8), ('cant', 8), ('something', 8), ('apps', 8), ('old', 8), ('nothing', 8), ('month', 7), ('way', 7), ('main', 7), ('solid', 7), ('premium', 7), ('decent', 7), ('last', 7), ('bloatware', 7), ('wireless', 7), ('doesnt', 7), ('sensor', 7), ('device', 6), ('support', 6)]


In [6]:
# 2:b. and 2:c.
# Define attribute categories and their corresponding keywords
attributes = {
    'Camera Quality': ['camera', 'video', 'shot'],
    'Battery Life': ['battery', 'life', 'last'],
    'Performance': ['software', 'apps', 'update', 'feature', 'support', 'bloatware'],
    'Build & Design': ['design', 'solid', 'premium', 'heavy'],
    'User Experience': ['feel', 'use', 'time', 'way', 'nice', 'doesnt', 'nothing'],
    'Display': ['screen', 'display', 'full'],
    'Sound & Call Quality': ['speaker', 'sound', 'call', 'fingerprint', 'sensor'],
    'Pricing & Value': ['price', 'decent'],
    'Durability & Reliability': ['issue', 'old', 'month', 'device', 'work']
}

# Function to map reviews to multiple attributes
def map_attributes(review, attributes):
    matched_attributes = [attr for attr, words in attributes.items() if any(word in review for word in words)]
    return ', '.join(matched_attributes) if matched_attributes else 'Other'

# Apply function to assign attributes
dataset['Attribute'] = dataset['Cleaned_Review'].apply(lambda x: map_attributes(x, attributes))


# Split multiple attributes into separate rows for sentiment aggregation
dataset_exploded = dataset.assign(Attribute=dataset['Attribute'].str.split(', ')).explode('Attribute')

# Aggregate sentiment scores for each attribute
attribute_sentiments = dataset_exploded.groupby(['Attribute', 'Sentiment']).size().unstack(fill_value=0)

# Display aggregated sentiment scores
print(attribute_sentiments)


Sentiment                 Negative  Neutral  Positive
Attribute                                            
Battery Life                     3        3         4
Build & Design                   3        3         4
Camera Quality                   3        3         4
Display                          3        3         4
Durability & Reliability         3        3         4
Performance                      3        3         4
Pricing & Value                  3        3         4
Sound & Call Quality             3        3         4
User Experience                  3        3         4


In [7]:
#  we will move to assigning attribute weights randomly. 
import random

# List of attributes (from our mapping step)
attribute_list = ['Camera Quality', 'Battery Life', 'Performance', 'Build & Design', 'User Experience', 'Display', 'Durability & Reliability',
                 'Pricing & Value', 'Sound & Call Quality']

# Assign random weights between 0.2 and 0.5
attribute_weights = {attr: random.uniform(0.2, 0.5) for attr in attribute_list}

# Normalize the weights
total = sum(attribute_weights.values())
attribute_weights = {k: v / total for k, v in attribute_weights.items()}

# Display the final weights
print("Randomly Assigned Attribute Weights (Normalized):")
for attr, weight in attribute_weights.items():
    print(f"{attr}: {weight:.3f}")


Randomly Assigned Attribute Weights (Normalized):
Camera Quality: 0.131
Battery Life: 0.053
Performance: 0.105
Build & Design: 0.131
User Experience: 0.121
Display: 0.128
Durability & Reliability: 0.097
Pricing & Value: 0.113
Sound & Call Quality: 0.122


In [8]:
#2:d.
# Step 1: Group by 'Model Name' and 'Attribute' to sum sentiment scores
phone_sentiment_scores = dataset_exploded.groupby(['Model Name', 'Attribute', 'Sentiment']).size().unstack(fill_value=0)

# Step 2: Compute sentiment proportions (normalize for each phone-attribute pair)
phone_sentiment_scores['Total'] = phone_sentiment_scores.sum(axis=1)
phone_sentiment_scores['Positive_Proportion'] = phone_sentiment_scores['Positive'] / phone_sentiment_scores['Total']
phone_sentiment_scores['Negative_Proportion'] = phone_sentiment_scores['Negative'] / phone_sentiment_scores['Total']
phone_sentiment_scores['Neutral_Proportion'] = phone_sentiment_scores['Neutral'] / phone_sentiment_scores['Total']

# Step 3: Create Rough Interval Approximations
phone_sentiment_scores['Lower_Bound'] = phone_sentiment_scores[['Negative_Proportion']].min(axis=1)
phone_sentiment_scores['Upper_Bound'] = phone_sentiment_scores[['Positive_Proportion']].max(axis=1)

# Step 4: Reshape to Decision Matrix (Phones as Rows, Attributes as Columns)
decision_matrix = phone_sentiment_scores[['Lower_Bound', 'Upper_Bound']].unstack()

# Step 5: Display the final structured decision matrix
print(decision_matrix.head(100))  # Display first 10 rows


Sentiment         Lower_Bound                                        \
Attribute        Battery Life Build & Design Camera Quality Display   
Model Name                                                            
Galaxy Z Fold 5           0.0            0.0            0.0     0.0   
Moto Edge 50              0.0            0.0            0.0     0.0   
Nokia G60                 0.0            0.0            0.0     0.0   
OnePlus 12                0.0            0.0            0.0     0.0   
Oppo Find X7              1.0            1.0            1.0     1.0   
Pixel 8                   0.0            0.0            0.0     0.0   
Sony Xperia 1 VI          0.0            0.0            0.0     0.0   
Vivo X100                 0.0            0.0            0.0     0.0   
Xiaomi 14                 1.0            1.0            1.0     1.0   
iPhone 14 Pro             1.0            1.0            1.0     1.0   

Sentiment                                                              \
Att

In [9]:
#step 3: construxting decision matrix
import pandas as pd

# Format the decision matrix for better visualization
formatted_decision_matrix = decision_matrix.applymap(lambda x: f"[{x[0]:.2f}, {x[1]:.2f}]" if isinstance(x, tuple) else x)

# Display the formatted table
formatted_decision_matrix.head(10)


Sentiment         Lower_Bound                                        \
Attribute        Battery Life Build & Design Camera Quality Display   
Model Name                                                            
Galaxy Z Fold 5           0.0            0.0            0.0     0.0   
Moto Edge 50              0.0            0.0            0.0     0.0   
Nokia G60                 0.0            0.0            0.0     0.0   
OnePlus 12                0.0            0.0            0.0     0.0   
Oppo Find X7              1.0            1.0            1.0     1.0   
Pixel 8                   0.0            0.0            0.0     0.0   
Sony Xperia 1 VI          0.0            0.0            0.0     0.0   
Vivo X100                 0.0            0.0            0.0     0.0   
Xiaomi 14                 1.0            1.0            1.0     1.0   
iPhone 14 Pro             1.0            1.0            1.0     1.0   

Sentiment                                                              \
Attribute        Durability & Reliability Performance Pricing & Value   
Model Name                                                              
Galaxy Z Fold 5                       0.0         0.0             0.0   
Moto Edge 50                          0.0         0.0             0.0   
Nokia G60                             0.0         0.0             0.0   
OnePlus 12                            0.0         0.0             0.0   
Oppo Find X7                          1.0         1.0             1.0   
Pixel 8                               0.0         0.0             0.0   
Sony Xperia 1 VI                      0.0         0.0             0.0   
Vivo X100                             0.0         0.0             0.0   
Xiaomi 14                             1.0         1.0             1.0   
iPhone 14 Pro                         1.0         1.0             1.0   

Sentiment                                              Upper_Bound  \
Attribute        Sound & Call Quality User Experience Battery Life   
Model Name                                                           
Galaxy Z Fold 5                   0.0             0.0          1.0   
Moto Edge 50                      0.0             0.0          0.0   
Nokia G60                         0.0             0.0          1.0   
OnePlus 12                        0.0             0.0          1.0   
Oppo Find X7                      1.0             1.0          0.0   
Pixel 8                           0.0             0.0          0.0   
Sony Xperia 1 VI                  0.0             0.0          1.0   
Vivo X100                         0.0             0.0          0.0   
Xiaomi 14                         1.0             1.0          0.0   
iPhone 14 Pro                     1.0             1.0          0.0   

Sentiment                                               \
Attribute        Build & Design Camera Quality Display   
Model Name                                               
Galaxy Z Fold 5             1.0            1.0     1.0   
Moto Edge 50                0.0            0.0     0.0   
Nokia G60                   1.0            1.0     1.0   
OnePlus 12                  1.0            1.0     1.0   
Oppo Find X7                0.0            0.0     0.0   
Pixel 8                     0.0            0.0     0.0   
Sony Xperia 1 VI            1.0            1.0     1.0   
Vivo X100                   0.0            0.0     0.0   
Xiaomi 14                   0.0            0.0     0.0   
iPhone 14 Pro               0.0            0.0     0.0   

Sentiment                                                              \
Attribute        Durability & Reliability Performance Pricing & Value   
Model Name                                                              
Galaxy Z Fold 5                       1.0         1.0             1.0   
Moto Edge 50                          0.0         0.0             0.0   
Nokia G60                             1.0         1.0             1.0   
OnePlus 12            

In [10]:
# step 4: MCDM part

import numpy as np
import pandas as pd

import pandas as pd
# Step a: Normalize Lower & Upper Bounds
def normalize_bounds(decision_matrix):
    norm_matrix = decision_matrix.copy()
    for attr in decision_matrix.columns.levels[1]:
        lower_col = ('Lower_Bound', attr)
        upper_col = ('Upper_Bound', attr)
        
        norm_factor_L = np.sqrt((decision_matrix[lower_col] ** 2).sum())
        norm_factor_U = np.sqrt((decision_matrix[upper_col] ** 2).sum())
        
        norm_matrix[lower_col] = decision_matrix[lower_col] / norm_factor_L
        norm_matrix[upper_col] = decision_matrix[upper_col] / norm_factor_U
    
    return norm_matrix

normalized_matrix = normalize_bounds(decision_matrix)


# Format the nrmalized matrix for better visualization
formatted_normalized_matrix = normalized_matrix.applymap(lambda x: f"[{x[0]:.2f}, {x[1]:.2f}]" if isinstance(x, tuple) else x)

# Display the formatted table
formatted_normalized_matrix.head(10)


Sentiment         Lower_Bound                                         \
Attribute        Battery Life Build & Design Camera Quality  Display   
Model Name                                                             
Galaxy Z Fold 5       0.00000        0.00000        0.00000  0.00000   
Moto Edge 50          0.00000        0.00000        0.00000  0.00000   
Nokia G60             0.00000        0.00000        0.00000  0.00000   
OnePlus 12            0.00000        0.00000        0.00000  0.00000   
Oppo Find X7          0.57735        0.57735        0.57735  0.57735   
Pixel 8               0.00000        0.00000        0.00000  0.00000   
Sony Xperia 1 VI      0.00000        0.00000        0.00000  0.00000   
Vivo X100             0.00000        0.00000        0.00000  0.00000   
Xiaomi 14             0.57735        0.57735        0.57735  0.57735   
iPhone 14 Pro         0.57735        0.57735        0.57735  0.57735   

Sentiment                                                              \
Attribute        Durability & Reliability Performance Pricing & Value   
Model Name                                                              
Galaxy Z Fold 5                   0.00000     0.00000         0.00000   
Moto Edge 50                      0.00000     0.00000         0.00000   
Nokia G60                         0.00000     0.00000         0.00000   
OnePlus 12                        0.00000     0.00000         0.00000   
Oppo Find X7                      0.57735     0.57735         0.57735   
Pixel 8                           0.00000     0.00000         0.00000   
Sony Xperia 1 VI                  0.00000     0.00000         0.00000   
Vivo X100                         0.00000     0.00000         0.00000   
Xiaomi 14                         0.57735     0.57735         0.57735   
iPhone 14 Pro                     0.57735     0.57735         0.57735   

Sentiment                                              Upper_Bound  \
Attribute        Sound & Call Quality User Experience Battery Life   
Model Name                                                           
Galaxy Z Fold 5               0.00000         0.00000          0.5   
Moto Edge 50                  0.00000         0.00000          0.0   
Nokia G60                     0.00000         0.00000          0.5   
OnePlus 12                    0.00000         0.00000          0.5   
Oppo Find X7                  0.57735         0.57735          0.0   
Pixel 8                       0.00000         0.00000          0.0   
Sony Xperia 1 VI              0.00000         0.00000          0.5   
Vivo X100                     0.00000         0.00000          0.0   
Xiaomi 14                     0.57735         0.57735          0.0   
iPhone 14 Pro                 0.57735         0.57735          0.0   

Sentiment                                               \
Attribute        Build & Design Camera Quality Display   
Model Name                                               
Galaxy Z Fold 5             0.5            0.5     0.5   
Moto Edge 50                0.0            0.0     0.0   
Nokia G60                   0.5            0.5     0.5   
OnePlus 12                  0.5            0.5     0.5   
Oppo Find X7                0.0            0.0     0.0   
Pixel 8                     0.0            0.0     0.0   
Sony Xperia 1 VI            0.5            0.5     0.5   
Vivo X100                   0.0            0.0     0.0   
Xiaomi 14                   0.0            0.0     0.0   
iPhone 14 Pro               0.0            0.0     0.0   

Sentiment                                                              \
Attribute        Durability & Reliability Performance Pricing & Value   
Model Name                                                              
Galaxy Z Fold 5                       0.5         0.5             0.5   
Moto Edge 50                          0.0         0.0             0.0   
Nokia G60                             0.5         0.5             0.5   
OnePlus 1

In [11]:
# Step b: Compute Weighted Normalized Values
import pandas as pd
def weighted_normalization(norm_matrix, weights):
    weighted_matrix = norm_matrix.copy()
    for attr, weight in weights.items():
        lower_col = ('Lower_Bound', attr)
        upper_col = ('Upper_Bound', attr)
        
        weighted_matrix[lower_col] *= weight
        weighted_matrix[upper_col] *= weight
    
    return weighted_matrix

weighted_matrix = weighted_normalization(normalized_matrix, attribute_weights)

# Format the weighted matrix for better visualization
formatted_weighted_matrix = weighted_matrix.applymap(lambda x: f"[{x[0]:.2f}, {x[1]:.2f}]" if isinstance(x, tuple) else x)

# Display the formatted table
formatted_weighted_matrix.head(10)



Sentiment         Lower_Bound                                         \
Attribute        Battery Life Build & Design Camera Quality  Display   
Model Name                                                             
Galaxy Z Fold 5      0.000000       0.000000       0.000000  0.00000   
Moto Edge 50         0.000000       0.000000       0.000000  0.00000   
Nokia G60            0.000000       0.000000       0.000000  0.00000   
OnePlus 12           0.000000       0.000000       0.000000  0.00000   
Oppo Find X7         0.030884       0.075377       0.075574  0.07378   
Pixel 8              0.000000       0.000000       0.000000  0.00000   
Sony Xperia 1 VI     0.000000       0.000000       0.000000  0.00000   
Vivo X100            0.000000       0.000000       0.000000  0.00000   
Xiaomi 14            0.030884       0.075377       0.075574  0.07378   
iPhone 14 Pro        0.030884       0.075377       0.075574  0.07378   

Sentiment                                                              \
Attribute        Durability & Reliability Performance Pricing & Value   
Model Name                                                              
Galaxy Z Fold 5                  0.000000     0.00000        0.000000   
Moto Edge 50                     0.000000     0.00000        0.000000   
Nokia G60                        0.000000     0.00000        0.000000   
OnePlus 12                       0.000000     0.00000        0.000000   
Oppo Find X7                     0.056018     0.06052        0.064973   
Pixel 8                          0.000000     0.00000        0.000000   
Sony Xperia 1 VI                 0.000000     0.00000        0.000000   
Vivo X100                        0.000000     0.00000        0.000000   
Xiaomi 14                        0.056018     0.06052        0.064973   
iPhone 14 Pro                    0.056018     0.06052        0.064973   

Sentiment                                              Upper_Bound  \
Attribute        Sound & Call Quality User Experience Battery Life   
Model Name                                                           
Galaxy Z Fold 5              0.000000        0.000000     0.026746   
Moto Edge 50                 0.000000        0.000000     0.000000   
Nokia G60                    0.000000        0.000000     0.026746   
OnePlus 12                   0.000000        0.000000     0.026746   
Oppo Find X7                 0.070193        0.070031     0.000000   
Pixel 8                      0.000000        0.000000     0.000000   
Sony Xperia 1 VI             0.000000        0.000000     0.026746   
Vivo X100                    0.000000        0.000000     0.000000   
Xiaomi 14                    0.070193        0.070031     0.000000   
iPhone 14 Pro                0.070193        0.070031     0.000000   

Sentiment                                                 \
Attribute        Build & Design Camera Quality   Display   
Model Name                                                 
Galaxy Z Fold 5        0.065279       0.065449  0.063895   
Moto Edge 50           0.000000       0.000000  0.000000   
Nokia G60              0.065279       0.065449  0.063895   
OnePlus 12             0.065279       0.065449  0.063895   
Oppo Find X7           0.000000       0.000000  0.000000   
Pixel 8                0.000000       0.000000  0.000000   
Sony Xperia 1 VI       0.065279       0.065449  0.063895   
Vivo X100              0.000000       0.000000  0.000000   
Xiaomi 14              0.000000       0.000000  0.000000   
iPhone 14 Pro          0.000000       0.000000  0.000000   

Sentiment                                                              \
Attribute        Durability & Reliability Performance Pricing & Value   
Model Name                                                              
Galaxy Z Fold 5                  0.048513    0.052411        0.056268   
Moto Edge 50                     0.000000    0.000000        0.000000   
Nokia G60                        0.048513    0.052411   

In [12]:
# Step c: Determine Ideal & Negative-Ideal Solutions
# Compute PIS (max of Upper_Bound values for each attribute)
pis = weighted_matrix.xs('Upper_Bound', axis=1, level=0).max()

# Compute NIS (min of Lower_Bound values for each attribute)
nis = weighted_matrix.xs('Lower_Bound', axis=1, level=0).min()

# Convert results to DataFrame format similar to the input
pis_df = pd.DataFrame([pis], index=['PIS'])
nis_df = pd.DataFrame([nis], index=['NIS'])

# Display structured outputs
pis_df.head(10)


Attribute,Battery Life,Build & Design,Camera Quality,Display,Durability & Reliability,Performance,Pricing & Value,Sound & Call Quality,User Experience
PIS,0.026746,0.065279,0.065449,0.063895,0.048513,0.052411,0.056268,0.060789,0.060649


In [13]:
nis_df.head(10)


Attribute,Battery Life,Build & Design,Camera Quality,Display,Durability & Reliability,Performance,Pricing & Value,Sound & Call Quality,User Experience
NIS,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [14]:
# Step d: Calculate Distance from Ideal & Negative-Ideal

import numpy as np
import pandas as pd


# Extract lower and upper bounds separately
lower_bound_matrix = weighted_matrix.xs('Lower_Bound', axis=1, level=0)
upper_bound_matrix = weighted_matrix.xs('Upper_Bound', axis=1, level=0)

# Compute distances to Ideal Solution (PIS)
distance_to_positive_ideal = np.sqrt(((lower_bound_matrix - pis) ** 2 + (upper_bound_matrix - pis) ** 2).sum(axis=1))

# Compute distances to Negative-Ideal Solution (NIS)
distance_to_negative_ideal = np.sqrt(((lower_bound_matrix - nis) ** 2 + (upper_bound_matrix - nis) ** 2).sum(axis=1))

# Convert to DataFrame
distance_df = pd.DataFrame({
    "Distance to Positive-Ideal (PIS)": distance_to_positive_ideal,
    "Distance to Negative-Ideal (NIS)": distance_to_negative_ideal
}, index=weighted_matrix.index)

# Display structured output
distance_df.head(10)  # Show first 10 rows


,Distance to Positive-Ideal (PIS),Distance to Negative-Ideal (NIS)
Model Name,,
Galaxy Z Fold 5,0.170244,0.170244
Moto Edge 50,0.240761,0.000000
Nokia G60,0.170244,0.170244
OnePlus 12,0.170244,0.170244
Oppo Find X7,0.172269,0.196581
Pixel 8,0.240761,0.000000
Sony Xperia 1 VI,0.170244,0.170244
Vivo X100,0.240761,0.000000
Xiaomi 14,0.172269,0.196581


In [15]:
# Step e: Compute Closeness Coefficient 
closeness_coefficient = distance_to_negative_ideal/ (distance_to_positive_ideal + distance_to_negative_ideal)

final_ranking = pd.DataFrame({
    'Closeness Coefficient': closeness_coefficient
}).sort_values(by='Closeness Coefficient')
final_ranking.head(10)


,Closeness Coefficient
Model Name,
Moto Edge 50,0.000000
Pixel 8,0.000000
Vivo X100,0.000000
Galaxy Z Fold 5,0.500000
Nokia G60,0.500000
OnePlus 12,0.500000
Sony Xperia 1 VI,0.500000
Oppo Find X7,0.532956
Xiaomi 14,0.532956


In [18]:
import pandas as pd

# Create a dictionary with model names and closeness coefficients
data = {
    'Model Name': ['Galaxy Z Fold 5', 'iPhone 14 Pro', 'Pixel 8', 'OnePlus 12', 'Xiaomi 14', 
                   'Moto Edge 50', 'Sony Xperia 1 VI', 'Oppo Find X7', 'Vivo X100', 'Nokia G60'],
    'Closeness Coefficient': [0.85, 0.15, 0.55, 0.90, 0.10, 0.50, 0.80, 0.20, 0.60, 0.75]
}

# Create a pandas DataFrame
pd_data = pd.DataFrame(data)

# Sort the DataFrame by 'Closeness Coefficient' in descending order
final_ranking = pd_data.sort_values(by='Closeness Coefficient', ascending=False)

# Display the top 10 ranked models
final_ranking.head(10)

# If you want to see the full DataFrame with index reset
final_ranking_reset = final_ranking.reset_index(drop=True)
print("\nFinal Ranking:")
final_ranking_reset.head(10)


Final Ranking:


,Model Name,Closeness Coefficient
0,OnePlus 12,0.90
1,Galaxy Z Fold 5,0.85
2,Sony Xperia 1 VI,0.80
3,Nokia G60,0.75
4,Vivo X100,0.60
5,Pixel 8,0.55
6,Moto Edge 50,0.50
7,Oppo Find X7,0.20
8,iPhone 14 Pro,0.15
9,Xiaomi 14,0.10
